In [25]:

!sudo apt install -y xvfb ffmpeg
!pip install gym-contra
!pip install 'imageio=2.4.0'
!pip install PILLOW
!pip install pyglet
!pip install pyvirtualdisplay
!pip install nes_py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.7).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
ERROR: Invalid requirement: 'imageio=2.4.0'
Hint: = is not a valid operator. Did you mean == ?


In [26]:
from __future__ import absolute_import, division, print_function
import gym
from gym.wrappers import Monitor
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import io
import glob
import numpy as np
import PIL.Image
import pyvirtualdisplay
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
!pip install tensorflow==1.14

In [27]:
display = Display(visible=0, size=(1400, 900))
display.start()

In [28]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [29]:
from Contra.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
from nes_py.wrappers import JoypadSpace


In [30]:
env_name = 'Contra-v0'
env  = gym.make(env_name)
env = JoypadSpace(env,SIMPLE_MOVEMENT)
env = wrap_env(env)
np.random.seed(123)
env.seed(123)

Make_done


[123]

In [31]:
print(env.action_space)
print(env.observation_space)

Discrete(7)
Box(0, 255, (240, 256, 3), uint8)


In [32]:
!pip install q keras==2.3.0
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Dropout
from keras.optimizers import Adam
import keras.backend as K

In [33]:
!pip install keras-rl==0.4.2

In [34]:
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [35]:
nb_actions = env.action_space.n

In [36]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dropout(0.2))

initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=0.7)
model.add(Dropout(0.2))

model.add(Dense(50,activation='relu'))
model.add(Dense(25,activation='sigmoid'))

model.add(Dense(nb_actions,activation='relu'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [37]:
memory = SequentialMemory(limit=100000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.fit(env, nb_steps=100000, visualize=True, verbose=2)

Training for 100000 steps ...

Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


Score 0
Score 0
Score 0
Score 0
Score 0


/usr/local/lib/python3.6/dist-packages/Contra/env_contra.py:153: RuntimeWarning: overflow encountered in ubyte_scalars
  _reward = self._x_position - self._x_position_last


Streaming output truncated to the last 5000 lines.
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score 7
Score

In [ ]:
dqn.save_weights('dqn_{}_100000_Bolztmann_weights.h5f'.format(env_name), overwrite=True)

In [ ]:
env.close()

In [ ]:
show_video()